## Análise jurimétrica express

Neste notebook, vamos usar o modelo de linguagem GPT-4.1-nano da OpenAI para classificar documentos em categorias. O GPT-4.1-nano é um modelo de linguagem de última geração que pode ser usado para uma variedade de tarefas de processamento de linguagem natural, incluindo classificação de texto.

Começamos carregando o modelo GPT-4.1-nano pré-treinado da OpenAI usando a biblioteca `openai` e, em seguida, usamos o modelo para classificar documentos em categorias. Para isso, fornecemos ao modelo um documento de entrada e ele nos dá uma lista de categorias possíveis para o documento.

Para acessar o modelo GPT-4.1-nano, você precisará de uma chave de API da OpenAI. Esse notebook já disponibiliza uma chave de API para você usar, mas você também pode obter sua própria chave de API se desejar. Lembre-se que liberar a chave de API é uma prática insegura, então não estamos fazendo isso pelo fluxo ideal.

In [19]:
# chave liberada para uso até o final do semestre, com algumas restrições
# carregando a biblioteca da openai
from openai import OpenAI
import pandas as pd
import dotenv
import os
import json
from pydantic import BaseModel, Field, ConfigDict
from tqdm import tqdm

_ = dotenv.load_dotenv(dotenv_path='.env', override=True)

key = os.getenv('OPENAI_API_KEY') # chave da openai

client = OpenAI(api_key=key)

In [ ]:
import juscraper as jus

tjsp = jus.scraper('tjsp')

result = tjsp.cjpg('inclusão indevida cadastro inadimplentes', paginas = range(0, 5))

In [ ]:
result

In [ ]:
result.value_counts("classe")

In [ ]:
print(result['decisao'].iloc[1])

## Desenvolvendo prompts para anotação de documentos jurídicos

A anotação automática de documentos jurídicos utilizando modelos de linguagem aumenta significativamente a eficiência na anotação de documentos. No entanto, a qualidade dos resultados pode variar dependendo da formulação do prompt utilizado.

Trata-se de uma área em desenvolvimento, então ainda não temos uma resposta final sobre como formular os prompts. No entanto, algumas dicas podem ser úteis:

1. **Escreva instruções claras e objetivas**: O prompt deve ser o mais claro e específico possível. Experimente verificar com você ou outra pessoa se as instruções são compreensíveis e diretas.

2. **Deixe claro quem é o assistente e qual seu papel**: O modelo de linguagem é um assistente, e é importante deixar claro qual é o papel dele na tarefa. Por exemplo, "você é um assistente jurídico que realiza a análise de petições iniciais em processos cíveis, resumindo as informações mais importantes para posterior avaliação de profissionais da advocacia".

3. **Descreva o que o modelo deve esperar de entrada**: Sempre que necessário, forneça contexto relevante ao modelo. Isso pode incluir a descrição do tipo de documento, o objetivo da anotação e as características específicas do caso em questão. Por exemplo, uma petição inicial de um processo é diferente de um tweet.

4. **Especifique o formato de saída**: Indicar o formato exato da resposta esperada é uma boa prática. Geralmente, pediremos para que a resposta seja em um JSON, e temos um parâmetro para forçar esse comportamento no modelo.

5. **Inclua exemplos**: Fornecer exemplos no prompt ajuda o modelo a entender o formato e o conteúdo esperado na resposta. O nome técnico disso é "*one shot*" ou "*few shot*" *learning*.

6. **Teste com alguns casos**: Sempre verifique se o prompt está funcionando com alguns casos antes de rodar para uma amostra grande, e faça ajustes caso necessário.

7. **Pedir um resumo**: Para tarefas complexas, pode ser útil pedir para o modelo de linguagem resumir a resposta em um parágrafo ou algumas frases, **antes** de realizar uma anotação em categorias. Isso pode aumentar a qualidade da resposta final.


**Obs**: você pode usar o ChatGPT para ajudar na criação de prompts. Isso economiza tempo e cria prompts eficazes. 

In [11]:
prompt = """
Você é um analista especializado em análise de processos judiciais relacionados a indenização por danos morais em processos de inclusão indevida em cadastro de inadimplentes.

Você deve extrair as informações abaixo da sentença, de forma estruturada.

"""

In [14]:
class Variaveis(BaseModel):
    model_config = ConfigDict(extra='forbid')
    data_decisao: str = Field(..., description="Data da decisão em formato YYYY-MM-DD.")
    magistrado: str = Field(..., description="Nome do juiz/juiza.")
    resultado_acao: str = Field(..., description="Resultado da decisão", enum= ["improcedente", "parcialmente procedente", "procedente"])
    valor_envolvido: float = Field(..., description="Valor envolvido na decisão.")
    indenizacao: float = Field(..., description="Valor da indenização.")

def classificar_decisao_df(prompt, julgado):
    resp_format = {
        "format": {"type": "json_schema", "name": "variaveis", "schema": Variaveis.model_json_schema(), "strict": True}
    }
    completion = client.responses.create(
        model="gpt-4.1-nano",
        instructions=prompt,
        input=julgado,
        temperature=0.0,
        text=resp_format
    )
    return json.loads(completion.output_text)

In [ ]:
classificar_decisao_df(prompt, result['decisao'].iloc[1])

In [ ]:
todos_resultados = []
for decisao in tqdm(result['decisao']):
  todos_resultados.append(classificar_decisao_df(prompt, decisao))

In [ ]:
df_final = pd.DataFrame(todos_resultados)
df_final

In [ ]:
df_final[df_final['indenizacao'] > 0]['indenizacao'].mean()

In [ ]:
df_final.value_counts("resultado_acao", normalize=True).reset_index()